Social media scraping
===

In [12]:
import os
import json

data_dir = "data/twitter"

if os.path.exists(data_dir) == False:
    os.makedirs(data_dir)

## Tweets by User

Scrape the last 100 tweets from `@derstandard` and store in json file

In [30]:
!snscrape --jsonl --max-results 100 twitter-user derstandardat > data/twitter/user-derstandardat.json

In [38]:
with open(data_dir+"/user-derstandardat.json", "r") as fo:
    derstandard_tweets = fo.readlines()

derstandard_tweets = [json.loads(derstandard_tweets[i]) for i in range(0, len(derstandard_tweets))]


for i in range(25):

    print(derstandard_tweets[i]["date"], derstandard_tweets[i]["rawContent"], derstandard_tweets[i]["hashtags"])

2023-01-22T18:02:02+00:00 Weiterarbeiten im Pensionsalter: "Meine Arbeit ist mein Leben":  https://t.co/C1zpXtuZ9j None
2023-01-22T18:02:01+00:00 Ältere sollen die Joblücke füllen. Aber wer will sie wirklich haben?:  https://t.co/a0yvyd7NAQ None
2023-01-22T17:31:23+00:00 Julian Schütter: Kitzbühel-Opfer und Skifahrer for Future:  https://t.co/YEY11CcFhs None
2023-01-22T17:31:22+00:00 Zum Jahrestag klingelt das FBI an Bidens Tür:  https://t.co/tNpiqpHV6R None
2023-01-22T17:31:21+00:00 Kindesmissbrauch: Mehr Mittel für Polizei:  https://t.co/4bTDKfCe0L None
2023-01-22T17:14:44+00:00 Sudoku mittel 5422a:  https://t.co/H3g7zKUt9u None
2023-01-22T17:14:43+00:00 Musk soll 80 Prozent der Belegschaft von Twitter gekündigt haben:  https://t.co/kISWrtnWt5 None
2023-01-22T17:14:42+00:00 Israels Überlebenskünstler:  https://t.co/rnz6TJzxqk None
2023-01-22T17:01:18+00:00 Volksoper in der Blödelmaschine: Offenbachs "Orpheus" als Slapstick-Operette:  https://t.co/PvKGOfOqiv None
2023-01-22T17:01:18+0

## Tweets by Hashtag

Scrape the last 100 tweets with `#chinesevirus` and store in txt file

In [39]:
!snscrape --jsonl --max-results 100 twitter-hashtag chinesevirus > data/twitter/hashtag-chinesevirus.json

In [40]:
with open(data_dir+"/hashtag-chinesevirus.json", "r") as fo:
    tweets = fo.readlines()

tweets = [json.loads(tweets[i]) for i in range(0, len(tweets))]


for i in range(25):

    print(tweets[i]["date"], tweets[i]["rawContent"], tweets[i]["hashtags"])

2023-01-22T18:41:36+00:00 #chineseVirus ['chineseVirus']
2023-01-22T15:30:19+00:00 #ChineseVirus update https://t.co/fkpN3sRSrL ['ChineseVirus']
2023-01-22T15:02:11+00:00 @AmbassadeChine Year of Chinese BAT
#chinesevirus ['chinesevirus']
2023-01-22T13:10:58+00:00 #ChineseNewYear #ChineseVirus ['ChineseNewYear', 'ChineseVirus']
2023-01-22T06:16:45+00:00 Fantastic. Instead of getting pissed off, it would be good to think about the points being brought up here with an open mind. You have been lied to.

#CovidVaccines
#VaccineDeath
#vaccine
#vaccinated
#covid
#covid19
#chinesevirus ['CovidVaccines', 'VaccineDeath', 'vaccine', 'vaccinated', 'covid', 'covid19', 'chinesevirus']
2023-01-21T14:14:23+00:00 @Fart47404663 More than 5.51 billion Ppl worldwide have received vaccines against #ChineseVirus or above 71.8% of the world population, like #China insulting our spiritual leader HH the Dalai Lama is hurting Tibetans feelings, have little more respect https://t.co/gFraCtbvN6 #FreeTibet #France